In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import ttest_ind
from decimal import Decimal
from statsmodels.sandbox.stats.multicomp import multipletests
data='../data/'

def get_p_rna(muld):
    loss=muld[muld['dna_c']=='loss']['rna_cor']
    gain=muld[muld['dna_c']=='gain']['rna_cor']
    mid=muld[muld['dna_c']=='mid']['rna_cor']
    loss=ttest_ind(loss, mid, equal_var=False)[1]
    gain=ttest_ind(gain, mid, equal_var=False)[1]
    print('%.e' % Decimal(loss), '%.e' % Decimal(gain))

def get_p_dna(muld):    
    loss=muld[muld['rna_c']=='down']['cor']
    gain=muld[muld['rna_c']=='up']['cor']
    mid=muld[muld['rna_c']=='mid']['cor']
    loss=ttest_ind(loss, mid, equal_var=False)[1]
    gain=ttest_ind(gain, mid, equal_var=False)[1]
    print('%.e' % Decimal(loss), '%.e' % Decimal(gain))  

In [2]:
rna_all=pd.read_csv(data+'results/rna_cor.csv',index_col=0)
rna_all=rna_all.rename(columns={'cor':'rna_cor','p':'rna_p'})
rna_all.index=rna_all['gen']
rna_all.shape

(113280, 6)

In [3]:
%%time
man=pd.read_csv(data+'clean/man_e1.csv',index_col=0)
dna_all=pd.read_csv(data+'results/dna_cor.csv',index_col=0)
dna_all=man[['dis','ch','pos','gene']].join(dna_all,how='inner').sort_values(['ch','pos'])
dna_all=dna_all.sort_values(['ch','pos'])

<timed exec>:2: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 15.9 s, sys: 2.07 s, total: 18 s
Wall time: 18 s


In [42]:
# DNAm -> RNA
#DNam Gain NOT significant
#DNAm Loss, NOT significant
m=100; n=2000
for dna_dir in ['DNAm loss','DNAm gain']:
    pls=[]
    for cohort in ['MESA1','MESA2','PPMI']:
        rna=rna_all[rna_all['cohort']==cohort]
        rna=rna.rename(columns={'cor':'rna_cor','p':'rna_p'})
        dna=dna_all[dna_all['cohort']==cohort]
        mul=dna.merge(rna[['rna_cor','rna_p']],left_on=['gene'],right_index=True,how='inner')
        muld=mul[mul['dis'].abs()<n]    
        muld=muld.sort_values('cor').copy()
        muld['dnam_c']='all CpGs'
        if dna_dir=='DNAm gain':
            muld.loc[muld.index[-m:],'dnam_c']=dna_dir
        else:
            muld.loc[muld.index[:m],'dnam_c']=dna_dir
        pl=muld[['dnam_c','rna_cor','cohort']]
        pls.append(pl)
    pls=pd.concat(pls)
    pls.loc[pls['dnam_c']==dna_dir,'dot_plot']=pls.loc[pls['dnam_c']==dna_dir,'rna_cor']
fig2a=pls[['dnam_c','rna_cor','cohort','dot_plot']]
fig2a.to_csv(data+'figs/2a.csv')
fig2a.shape    

(817998, 4)

In [43]:
# RNA --> DNAm 
# Up regulated, NOT significant 
# Down regulated, SIGNIFICANT DNMm Gain
m=100; n=2000
for rna_dir in ['up-regulated','down-regulated']:
    pls=[]
    for cohort in ['MESA1','MESA2','PPMI']:
        rna=rna_all[rna_all['cohort']==cohort]
        dna=dna_all[(dna_all['cohort']==cohort)]
        mul=dna.merge(rna[['rna_cor','rna_p']],left_on=['gene'],right_index=True,how='inner')
        muld=mul[mul['dis'].abs()<n]    
        muld=muld.sort_values('rna_cor').copy()
        muld['rna_c']='all transcripts'
        if rna_dir=='down-regulated':
            muld.loc[muld.index[:m],'rna_c']=rna_dir
        else:
            muld.loc[muld.index[-m:],'rna_c']=rna_dir
        pl=muld[['rna_c','cor','cohort']]
        pls.append(pl)
pls=pd.concat(pls)
pls.loc[pls['rna_c']==rna_dir,'dot_plot']=pls.loc[pls['rna_c']==rna_dir,'cor']
fig2b=pls[['rna_c','cor','cohort','dot_plot']]
fig2b.to_csv(data+'figs/2b.csv')
fig2b.shape

(817998, 4)

In [37]:
m=100; n=2000
rnap=.05
pl=[]
cohorts=['MESA1','MESA2','PPMI']
for cohort in cohorts:
    ## all genes
    rna=rna_all[rna_all['cohort']==cohort]
    cor_sub=rna.copy()
    cor_sub['q']='Among all transcripts'
    cor_sub['pp']=(cor_sub['rna_cor']<0)&(cor_sub['rna_p']<rnap) #down            
    pl.append(cor_sub[['cohort','pp','q']])

    ## aging genes
    cor_sub=rna[rna['proposed']=='down'].copy()
    cor_sub['q']="Among Peters et al's aging transcripts"
    cor_sub['pp']=(cor_sub['rna_cor']<0)&(cor_sub['rna_p']<rnap) #down
    pl.append(cor_sub[['cohort','pp','q']])    
    for i in [1,2]: 
        q=10**(-i)
        dna=dna_all[dna_all['cohort']==cohort]
        dd=dna[dna['dis'].abs()<=n].copy() ### 2000 fromTSS
        dd['p']=(dd['p']<q)&(dd['cor']>0)              #DNAm gain
        dd['p+']=dd['p'].shift(1) 
        dd['p-']=dd['p'].shift(-1)
        dd['p3']=dd['p']&dd['p+']&dd['p-']
        ddp=dd[dd['p3']==True]
        cor_sub=rna[rna.index.isin(ddp['gene'])].copy()
        cor_sub['q']='Among aging trans. & aging CpGs (p='+str(q)+')'
        cor_sub=cor_sub[cor_sub['proposed']=='down']           #down
        cor_sub['pp']=(cor_sub['rna_cor']<0)&(cor_sub['rna_p']<rnap) #down
        pl.append(cor_sub[['cohort','pp','q']])   
pl=pd.concat(pl) 

In [19]:
%%time
fig2e=pl
fig2e.to_csv(data+'figs/2e.csv')
fig2e.shape

CPU times: user 160 ms, sys: 5.06 ms, total: 165 ms
Wall time: 327 ms


(59380, 3)